# CS329E Data Analytics Project

**Team Members:** *Bryce Holladay, Joshua Mathew, Austin Rinn, Eddie Castillo*

Using the techniques that we have learned in class, we attempted to predict the result of a National Football League (NFL) play based on elements existing before the play begins, such as field position and time remaining in game.

We used data collected from [publiclly available play by play data from the years 2013 through 2019](http://nflsavant.com/about.php) to build our model. As inputs, our model takes parameters of time, down, yards to go, yardline, and offensive formation. Our data has several play resultant classifiers that we have tried to predict, including touchdowns, interceptions, sacks, first downs, yards, and penalties.

In order to fit the data into our model, we performed several actions to pre-process it, including reformatting time into a linear format and removing non-descriptive data like season year. The results of our model are shown below.

In [118]:
# Use this cell for any notes
# Rubric: https://utexas.instructure.com/courses/1275914/assignments/4897667
import pandas as pd, numpy as np

## Data Preprocessing
Data cleaning, data exploration, and feature engineering

In [119]:
#Read in data from csv
#For building purposes use one season to save processing time.
#For final runs we will switch to compiled data sheet with all seasons.
#Display initial data head

df19 = pd.read_csv('pbp-2019.csv')
df19.head()

,GameId,GameDate,Quarter,Minute,Second,OffenseTeam,DefenseTeam,Down,ToGo,YardLine,...,IsTwoPointConversion,IsTwoPointConversionSuccessful,RushDirection,YardLineFixed,YardLineDirection,IsPenaltyAccepted,PenaltyTeam,IsNoPlay,PenaltyType,PenaltyYards
0,2019100605,2019-10-06,1,2,25,OAK,CHI,1,10,50,...,0,0,NaN,50,OPP,0,NaN,0,NaN,0
1,2019100605,2019-10-06,1,1,45,OAK,CHI,2,9,51,...,0,0,RIGHT GUARD,49,OPP,0,NaN,0,NaN,0
2,2019101400,2019-10-14,1,10,34,DET,GB,1,10,84,...,0,0,RIGHT TACKLE,16,OPP,0,NaN,0,NaN,0
3,2019101400,2019-10-14,1,9,55,DET,GB,2,9,85,...,0,0,NaN,15,OPP,0,NaN,0,NaN,0
4,2019101400,2019-10-14,1,9,10,DET,GB,3,3,91,...,0,0,NaN,9,OPP,0,NaN,0,NaN,0


In [120]:
#Convert time into a standard format
#Display both format heads for comparison
df19['AbsoluteTime'] = (df19['Quarter']-1)*900 + df19['Minute']*60 + df19['Second'] 

In [121]:
#Convert GameDate into just month to represent time of year
import re
pattern = "-(.*?)\-"
for index in range(df19.shape[0]):
   df19['GameDate'][index] = re.search(pattern, df19['GameDate'][index]).group(1)

C:\Users\Josh Mathew\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [122]:
df19.rename(columns={"GameDate": "GameMonth"})

,GameId,GameMonth,Quarter,Minute,Second,OffenseTeam,DefenseTeam,Down,ToGo,YardLine,...,IsTwoPointConversionSuccessful,RushDirection,YardLineFixed,YardLineDirection,IsPenaltyAccepted,PenaltyTeam,IsNoPlay,PenaltyType,PenaltyYards,AbsoluteTime
0,2019100605,10,1,2,25,OAK,CHI,1,10,50,...,0,NaN,50,OPP,0,NaN,0,NaN,0,145
1,2019100605,10,1,1,45,OAK,CHI,2,9,51,...,0,RIGHT GUARD,49,OPP,0,NaN,0,NaN,0,105
2,2019101400,10,1,10,34,DET,GB,1,10,84,...,0,RIGHT TACKLE,16,OPP,0,NaN,0,NaN,0,634
3,2019101400,10,1,9,55,DET,GB,2,9,85,...,0,NaN,15,OPP,0,NaN,0,NaN,0,595
4,2019101400,10,1,9,10,DET,GB,3,3,91,...,0,NaN,9,OPP,0,NaN,0,NaN,0,550
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42181,2019090803,09,3,7,54,BAL,MIA,0,0,35,...,0,NaN,35,OWN,0,NaN,0,NaN,0,2274
42182,2019090800,09,3,0,0,NaN,LA,0,0,0,...,0,NaN,0,OWN,0,NaN,0,NaN,0,1800
42183,2019090800,09,1,0,0,NaN,LA,0,0,0,...,0,NaN,0,OWN,0,NaN,0,NaN,0,0
42184,2019090500,09,3,15,0,GB,CHI,0,0,35,...,0,NaN,35,OWN,0,NaN,0,NaN,0,2700


In [123]:
#Purge other data not needed
# No longer need Quarter, Minute, Seconds
# GameID has no effect on the play
# SeriesFirstDown has no description
# NextScore is 0 for every row. Has no effect.
df_19 = df19.drop(['Quarter', 'Minute', 'Second', 'GameId', 'Unnamed: 10', 'Unnamed: 12', 'Unnamed: 16', 'Unnamed: 17', 'SeriesFirstDown', 'NextScore', 'TeamWin', 'Description', 'OffenseTeam', 'DefenseTeam', 'SeasonYear'], axis=1)


In [124]:
# Yards is information known after the play
df19_isTD = df_19.drop(['Yards', 'IsSack', 'IsChallenge', 'IsChallengeReversed', 'Challenger', 'IsMeasurement', 'IsInterception', 'IsFumble', 'IsPenalty', 'IsTwoPointConversion', 'IsTwoPointConversionSuccessful', 'IsPenaltyAccepted', 'PenaltyTeam', 'PenaltyType', 'PenaltyYards', 'YardLineFixed'], axis=1)
df19_isTD.head()

,GameDate,Down,ToGo,YardLine,Formation,PlayType,IsRush,IsPass,IsIncomplete,IsTouchdown,PassType,RushDirection,YardLineDirection,IsNoPlay,AbsoluteTime
0,10,1,10,50,NO HUDDLE,PASS,0,1,0,0,SHORT LEFT,NaN,OPP,0,145
1,10,2,9,51,UNDER CENTER,RUSH,1,0,0,0,NaN,RIGHT GUARD,OPP,0,105
2,10,1,10,84,UNDER CENTER,RUSH,1,0,0,0,NaN,RIGHT TACKLE,OPP,0,634
3,10,2,9,85,SHOTGUN,PASS,0,1,0,0,SHORT MIDDLE,NaN,OPP,0,595
4,10,3,3,91,SHOTGUN,PASS,0,1,0,0,SHORT MIDDLE,NaN,OPP,0,550


In [125]:


# Combine RushDirection and PassType to get one column with play type
# No need for PlayType column anymore because it says the same information but less descriptive
df19_isTD['RushDirection'] = df19_isTD['RushDirection'].fillna('')
df19_isTD['PassType'] = df19_isTD['PassType'].fillna('')
df19_isTD['PlayType2'] = df19_isTD['RushDirection'] + df19_isTD['PassType']
df19_isTD = df19_isTD.drop('PlayType', axis=1)


In [126]:
df19.rename(columns={"PlayType": "PlayType2"})
df19_isTD = df19_isTD.drop(['IsIncomplete', 'PassType', 'RushDirection', 'YardLineDirection'], axis=1)
df19_isTD.head(50)

,GameDate,Down,ToGo,YardLine,Formation,IsRush,IsPass,IsTouchdown,IsNoPlay,AbsoluteTime,PlayType2
0,10,1,10,50,NO HUDDLE,0,1,0,0,145,SHORT LEFT
1,10,2,9,51,UNDER CENTER,1,0,0,0,105,RIGHT GUARD
2,10,1,10,84,UNDER CENTER,1,0,0,0,634,RIGHT TACKLE
3,10,2,9,85,SHOTGUN,0,1,0,0,595,SHORT MIDDLE
4,10,3,3,91,SHOTGUN,0,1,0,0,550,SHORT MIDDLE
5,11,1,10,56,SHOTGUN,0,1,0,0,447,SHORT MIDDLE
6,11,1,10,78,SHOTGUN,0,1,0,0,404,DEEP LEFT
7,11,2,1,70,SHOTGUN,0,0,0,0,2049,
8,11,1,10,73,SHOTGUN,0,0,0,1,2008,
9,11,2,10,51,SHOTGUN,1,0,0,0,736,LEFT END


In [128]:

c = (df19_isTD['PlayType2'] == '').sum()
print(c)
df19_isTD.head(50)
df19_isTD.describe()

13334


,Down,ToGo,YardLine,IsRush,IsPass,IsTouchdown,IsNoPlay,AbsoluteTime
count,42186.000000,42186.000000,42186.000000,42186.000000,42186.000000,42186.000000,42186.000000,42186.000000
mean,1.666003,7.302209,45.411321,0.286778,0.415351,0.031503,0.058195,1818.090433
std,1.170637,4.989110,26.773955,0.452262,0.492788,0.174676,0.234114,1036.803255
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,3.000000,25.000000,0.000000,0.000000,0.000000,0.000000,932.000000
50%,1.000000,9.000000,40.000000,0.000000,0.000000,0.000000,0.000000,1800.000000
75%,2.000000,10.000000,66.000000,1.000000,1.000000,0.000000,0.000000,2761.000000
max,4.000000,40.000000,99.000000,1.000000,1.000000,1.000000,1.000000,4200.000000


In [130]:
# Get names of indexes for which plays are not rush or pass
indexNames = df19_isTD[(df19_isTD['IsRush'] == 0) & (df19_isTD['IsPass'] == 0)].index
 
# Delete these row indexes from dataFrame
df19_isTD.drop(indexNames , inplace=True)
df19_isTD.describe()

,Down,ToGo,YardLine,IsRush,IsPass,IsTouchdown,IsNoPlay,AbsoluteTime
count,29620.000000,29620.000000,29620.000000,29620.000000,29620.000000,29620.000000,29620.000000,29620.000000
mean,1.784571,8.660331,48.803916,0.408440,0.591560,0.043214,0.049865,1826.877819
std,0.819517,4.022085,24.384350,0.491554,0.491554,0.203342,0.217670,1040.954517
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,6.000000,28.000000,0.000000,0.000000,0.000000,0.000000,939.000000
50%,2.000000,10.000000,45.000000,0.000000,1.000000,0.000000,0.000000,1808.000000
75%,2.000000,10.000000,68.000000,1.000000,1.000000,0.000000,0.000000,2774.000000
max,4.000000,40.000000,99.000000,1.000000,1.000000,1.000000,1.000000,4200.000000


In [131]:
# Get names of indexes for which plays arre not specified
indexNames = df19_isTD[df19_isTD['PlayType2'] == ''].index
 
# Delete these row indexes from dataFrame
df19_isTD.drop(indexNames , inplace=True)

In [133]:
df19_isTD.head(100)

,GameDate,Down,ToGo,YardLine,Formation,IsRush,IsPass,IsTouchdown,IsNoPlay,AbsoluteTime,PlayType2
0,10,1,10,50,NO HUDDLE,0,1,0,0,145,SHORT LEFT
1,10,2,9,51,UNDER CENTER,1,0,0,0,105,RIGHT GUARD
2,10,1,10,84,UNDER CENTER,1,0,0,0,634,RIGHT TACKLE
3,10,2,9,85,SHOTGUN,0,1,0,0,595,SHORT MIDDLE
4,10,3,3,91,SHOTGUN,0,1,0,0,550,SHORT MIDDLE
...,...,...,...,...,...,...,...,...,...,...,...
131,12,1,10,76,UNDER CENTER,1,0,0,0,2334,RIGHT TACKLE
132,12,1,10,54,NO HUDDLE SHOTGUN,0,1,0,0,2365,SHORT MIDDLE
134,12,1,10,90,SHOTGUN,0,1,1,0,944,SHORT LEFT
135,12,3,3,85,SHOTGUN,0,1,0,0,968,SHORT RIGHT


In [138]:
#Label Encode
from sklearn.preprocessing import LabelEncoder
# creating initial dataframe
#bridge_types = ('Arch','Beam','Truss','Cantilever','Tied Arch','Suspension','Cable')
#bridge_df = pd.DataFrame(bridge_types, columns=['Bridge_Types'])
# creating instance of labelencoder
labelencoder = LabelEncoder()
# Assigning numerical values and storing in another column
df19_isTD['Formation_Code'] = labelencoder.fit_transform(df19_isTD['Formation'])
df19_isTD['PlayType_Code'] = labelencoder.fit_transform(df19_isTD['PlayType2'])


KeyError: "['Formation' 'PlayType2'] not found in axis"

In [141]:
df19_isTD_encoded = df19_isTD.drop(['Formation', 'PlayType2'], axis=1)

In [142]:
df19_isTD_encoded

,GameDate,Down,ToGo,YardLine,IsRush,IsPass,IsTouchdown,IsNoPlay,AbsoluteTime,Formation_Code,PlayType_Code
0,10,1,10,50,0,1,0,0,145,0,14
1,10,2,9,51,1,0,0,0,105,4,12
2,10,1,10,84,1,0,0,0,634,4,13
3,10,2,9,85,0,1,0,0,595,3,15
4,10,3,3,91,0,1,0,0,550,3,15
...,...,...,...,...,...,...,...,...,...,...,...
42149,09,2,5,42,1,0,0,0,1727,4,6
42152,09,1,10,22,1,0,0,0,3544,4,5
42154,09,1,15,40,1,0,0,0,2495,4,11
42159,09,1,10,79,0,1,0,0,1854,4,4


In [22]:
#Separate labels from classifiers
#Labels will most likely need to be converted into one column with casting as nothing=0, touchdown=1, interception=2, etc 

In [23]:
#Confirm and display final data

## Data Analysis

#### Decision Trees

In [24]:
#Perform Decision Trees (Assign 1)
#Report results, including accuracy scores and appropriate visuals

#### KNN

In [25]:
#Perform KNN (Assign 2)
#Report results, including accuracy scores and appropriate visuals

#### Naive-Bayes

In [26]:
#Perform Naive-Bayes (Assign 2)
#Report results, including accuracy scores and appropriate visuals

#### SVM

In [27]:
#Perform SVM (Assign 3)
#Report results, including accuracy scores and appropriate visuals

#### Neural Net

In [28]:
#Perform Neural Net (Assign 3)
#Report results, including accuracy scores and appropriate visuals

#### Ensembles

In [29]:
#Perform Ensembles (Assign 3)
#Report results, including accuracy scores and appropriate visuals

## Model Analysis

In [30]:
#Compare accuracy scores and other metrics for our different models.
#How confident are we in the success rates of these various models?

In [31]:
#Discuss which model was the best.

In [32]:
#Discuss data. What issues may have existed in the data?  What assumptions did we make? What could have made our data better?

In [33]:
#Discuss our project as a whole. How could we have improved project? How might this model be used in real world applications?